<a href="https://colab.research.google.com/github/cluciani-angel/grupo-angel-erp-core/blob/main/ZohoAUDIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import glob
import re
from datetime import datetime

# ==========================================
# CONFIGURACIÓN
# ==========================================
INPUT_FOLDER = '.'  # Carpeta actual en Colab
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")
OUTPUT_CSV_NAME = f'AUDITORIA_DETALLADA_{TIMESTAMP}.csv'
REPORT_FILE_NAME = f'MEMORANDO_TECNICO_MIGRACION_{TIMESTAMP}.md'

# Listas para acumular hallazgos y narrativa del reporte
audit_findings = []
report_log = []

def log_event(message, level="INFO"):
    """Registra eventos para el reporte ejecutivo"""
    icon = "✅" if level == "INFO" else ("⚠️" if level == "WARNING" else "🚨")
    entry = f"{icon} **{level}:** {message}"
    report_log.append(entry)
    print(entry)

# ==========================================
# 1. MOTOR DE DIAGNÓSTICO
# ==========================================

log_event("INICIANDO PROCESO DE INGENIERÍA DE DATOS GRUPO ANGEL", "INFO")
log_event(f"Escaneando archivos en {INPUT_FOLDER}...", "INFO")

csv_files = glob.glob(os.path.join(INPUT_FOLDER, "*.csv"))
log_event(f"Se detectaron {len(csv_files)} archivos para procesar.", "INFO")

for filename in csv_files:
    try:
        # Carga ligera inicial
        df = pd.read_csv(filename, encoding='utf-8', on_bad_lines='skip', low_memory=False)
        base_name = os.path.basename(filename)

        # --- A. AUDITORÍA DE ITEMS (PRODUCTOS/SERVICIOS) ---
        if {'Item Name', 'SKU', 'Product Type'}.issubset(df.columns):
            log_event(f"Analizando Catálogo de Productos: {base_name}", "INFO")

            # 1. Check SKUs Inválidos
            def check_sku(x):
                if pd.isna(x) or str(x).strip() == '': return 'MISSING'
                if not re.match(r'^[A-Z0-9-]+$', str(x)): return 'INVALID_FORMAT'
                return 'OK'

            df['SKU_CHECK'] = df['SKU'].apply(check_sku)
            bad_skus = df[df['SKU_CHECK'] != 'OK']

            if not bad_skus.empty:
                count = len(bad_skus)
                log_event(f"Detectados {count} Items con SKUs sucios o faltantes en {base_name}.", "WARNING")
                # Guardar detalle
                for _, row in bad_skus.iterrows():
                    audit_findings.append({
                        'Archivo': base_name,
                        'Tipo_Error': f'SKU_{row["SKU_CHECK"]}',
                        'Identificador': row['Item Name'],
                        'Valor_Actual': row['SKU'],
                        'Accion_Requerida': 'Sanitizar para SQL/Supabase',
                        'Impacto': 'Alto (Rompe Sync)'
                    })

            # 2. Check Servicio con Inventario (Error Contable)
            if 'Inventory Account Code' in df.columns:
                services_with_stock = df[
                    (df['Product Type'] == 'service') &
                    (df['Inventory Account Code'].astype(str).str.startswith('1-', na=False))
                ]
                if not services_with_stock.empty:
                    count = len(services_with_stock)
                    log_event(f"¡CRÍTICO! {count} Servicios configurados erróneamente con cuenta de Inventario.", "ERROR")
                    for _, row in services_with_stock.iterrows():
                        audit_findings.append({
                            'Archivo': base_name,
                            'Tipo_Error': 'CONTABILIDAD_MIXTA',
                            'Identificador': row['Item Name'],
                            'Valor_Actual': row['Inventory Account Code'],
                            'Accion_Requerida': 'Cambiar a Tipo Goods o Quitar Cuenta Activo',
                            'Impacto': 'Crítico (Infla Balance General)'
                        })

        # --- B. AUDITORÍA CHART OF ACCOUNTS (CoA) ---
        elif {'Account Code', 'Account Name'}.issubset(df.columns):
            log_event(f"Analizando Estructura Contable: {base_name}", "INFO")

            # Check Formato X-XXXX
            def check_coa(x):
                if pd.isna(x): return 'MISSING'
                if not re.match(r'^\d-\d{4}$', str(x).strip()): return 'BAD_FORMAT'
                return 'OK'

            df['COA_CHECK'] = df['Account Code'].apply(check_coa)
            bad_coa = df[df['COA_CHECK'] != 'OK']

            if not bad_coa.empty:
                log_event(f"{len(bad_coa)} Cuentas contables no cumplen el estándar X-XXXX.", "WARNING")
                for _, row in bad_coa.iterrows():
                    audit_findings.append({
                        'Archivo': base_name,
                        'Tipo_Error': 'COA_FORMATO',
                        'Identificador': row['Account Name'],
                        'Valor_Actual': row['Account Code'],
                        'Accion_Requerida': 'Renombrar Código para Consolidación',
                        'Impacto': 'Medio (Impide Reportes BI Unificados)'
                    })

    except Exception as e:
        log_event(f"Error procesando {filename}: {str(e)}", "ERROR")

# ==========================================
# 2. GENERACIÓN DE ENTREGABLES
# ==========================================

# A. CSV DE TRABAJO (Técnico)
if audit_findings:
    df_audit = pd.DataFrame(audit_findings)
    df_audit.to_csv(OUTPUT_CSV_NAME, index=False)
    log_event(f"Generado CSV técnico: {OUTPUT_CSV_NAME}", "INFO")
else:
    log_event("No se encontraron errores bloqueantes (¡Increíble!).", "INFO")

# B. REPORTE EJECUTIVO (Gerencial)
with open(REPORT_FILE_NAME, 'w', encoding='utf-8') as f:
    f.write(f"# MEMORANDO TÉCNICO: MIGRACIÓN ECOSISTEMA ZOHO - GRUPO ANGEL\n")
    f.write(f"**Fecha:** {datetime.now().strftime('%d/%m/%Y')}\n")
    f.write(f"**Responsable:** Arquitectura de Datos\n\n")

    f.write("## 1. OBJETIVO DEL PROCESO\n")
    f.write("Asegurar que la data de las 8 filiales sea consistente para permitir la **Consolidación Financiera Automática** y la sincronización con **Supabase/App Móvil**.\n\n")

    f.write("## 2. RESUMEN DE HALLAZGOS\n")
    f.write("Se han analizado los archivos CSV maestros detectando las siguientes inconsistencias que requieren acción:\n\n")
    for line in report_log:
        f.write(f"{line}\n\n")

    f.write("## 3. BENEFICIOS DE ESTA LIMPIEZA\n")
    f.write("* **Para Contabilidad:** Elimina el trabajo manual de 'cuadrar' reportes entre empresas. Unifica el Plan de Cuentas.\n")
    f.write("* **Para Gerencia:** Permite Dashboards en Tiempo Real en Zoho Analytics sin errores de datos.\n")
    f.write("* **Para Operaciones:** Evita errores en la App Móvil (Supabase) causados por SKUs con caracteres extraños.\n\n")

    f.write("## 4. SIGUIENTES PASOS\n")
    f.write(f"1. Descargar el archivo `{OUTPUT_CSV_NAME}`.\n")
    f.write("2. Abrirlo en Google Sheets.\n")
    f.write("3. Asignar responsables para corregir los items marcados como 'CRÍTICO'.\n")

print(f"\n✨ PROCESO TERMINADO. Archivos generados:\n 1. {OUTPUT_CSV_NAME} (Data)\n 2. {REPORT_FILE_NAME} (Reporte)")

✅ **INFO:** INICIANDO PROCESO DE INGENIERÍA DE DATOS GRUPO ANGEL
✅ **INFO:** Escaneando archivos en ....
✅ **INFO:** Se detectaron 0 archivos para procesar.
✅ **INFO:** No se encontraron errores bloqueantes (¡Increíble!).

✨ PROCESO TERMINADO. Archivos generados:
 1. AUDITORIA_DETALLADA_20251214_1744.csv (Data)
 2. MEMORANDO_TECNICO_MIGRACION_20251214_1744.md (Reporte)


# ==============================================================================
# 🏗️ SCRIPT DE AUDITORÍA Y GOBERNANZA DE DATOS - GRUPO ANGEL (MF WORLD)
# ==============================================================================
# AUTOR: ZBooks AI Architecture
# PROPÓSITO: Analizar lote masivo de CSVs de Zoho Books para migración a Supabase.
# SALIDA: 1. Excel con errores técnicos (Data Cleaning).
#         2. Reporte Markdown para Stakeholders (Gerencia/Contabilidad).
# ==============================================================================


In [ ]:
import pandas as pd
import os
import re
import glob
from datetime import datetime
from google.colab import drive

# 1️⃣ CONEXIÓN CON GOOGLE DRIVE
# ------------------------------------------------------------------------------
# Esto monta tu Drive como si fuera un disco duro local.
print("🔌 Conectando a Google Drive...")
drive.mount('/content/drive')

# 📂 CONFIGURACIÓN DE RUTAS (Ajusta si tu carpeta se llama diferente)
BASE_PATH = '/content/drive/My Drive/MIGRACION_GRUPO_ANGEL'
INPUT_PATH = os.path.join(BASE_PATH, 'INPUT_CSVS')
OUTPUT_PATH = BASE_PATH # Los reportes se guardarán en la raíz de la carpeta

# Verificación de seguridad
if not os.path.exists(INPUT_PATH):
    print(f"❌ ERROR: No encuentro la carpeta {INPUT_PATH}")
    print("   Por favor crea la carpeta en Drive y sube los CSVs ahí.")
else:
    print(f"✅ Carpeta encontrada. Iniciando escaneo en: {INPUT_PATH}")

# ==============================================================================
# 2️⃣ LÓGICA DE DETECCIÓN INTELIGENTE (CEREBRO DEL SCRIPT)
# ==============================================================================
# Esta función "mira" dentro de cada CSV para saber qué es (Factura, Item, Cuenta)
# sin importar el nombre del archivo.

def identify_file_type(df):
    cols = set(df.columns)
    # Huellas digitales de cada tipo de archivo en Zoho
    if {'Item Name', 'SKU', 'Product Type'}.issubset(cols): return 'ITEMS'
    if {'Account Code', 'Account Name'}.issubset(cols): return 'COA' # Chart of Accounts
    if {'Invoice Number', 'Customer Name'}.issubset(cols): return 'INVOICES'
    if {'Vendor Name', 'Bill Number'}.issubset(cols): return 'BILLS'
    return 'OTHER'

data_map = {}
all_files = glob.glob(os.path.join(INPUT_PATH, "*.csv"))
file_log = [] # Para el reporte

print(f"🔄 Procesando {len(all_files)} archivos encontrados...")

for filename in all_files:
    try:
        # Leemos solo cabeceras primero para velocidad
        df_preview = pd.read_csv(filename, nrows=2, encoding='utf-8', on_bad_lines='skip')
        ftype = identify_file_type(df_preview)

        if ftype != 'OTHER':
            # Carga completa si es un archivo reconocido
            df = pd.read_csv(filename, encoding='utf-8', on_bad_lines='skip', low_memory=False)
            data_map[ftype] = df
            file_log.append(f"- **{os.path.basename(filename)}**: Identificado como `{ftype}` ({len(df)} registros).")
            print(f"   -> {ftype}: {os.path.basename(filename)}")
    except Exception as e:
        print(f"   ⚠️ Error leyendo {filename}: {e}")

# ==============================================================================
# 3️⃣ REGLAS DE NEGOCIO Y AUDITORÍA (AQUÍ ESTÁ LA INTELIGENCIA)
# ==============================================================================

audit_findings = {}
stats = {'invalid_skus': 0, 'critical_errors': 0, 'coa_warnings': 0}

# --- A. AUDITORÍA DE ITEMS (SKU & TIPOLOGÍA) ---
if 'ITEMS' in data_map:
    df_items = data_map['ITEMS']

    # 1. Validación SKU (Solo Mayúsculas, Números y Guiones)
    # Explicación: SQL y Supabase odian espacios y caracteres especiales.
    def check_sku(sku):
        if pd.isna(sku) or str(sku).strip() == '': return 'FALTA_SKU'
        if not re.match(r'^[A-Z0-9-]+$', str(sku)): return 'CARACTER_ILEGAL'
        return 'OK'

    df_items['ESTADO_SKU'] = df_items['SKU'].apply(check_sku)

    # 2. Validación Cruzada: ¿Servicios moviendo Inventario? (RED FLAG)
    # Un servicio (mano de obra) nunca debe tener una cuenta de Activo (1-XXXX)
    df_items['ERROR_CRITICO'] = False
    if 'Inventory Account Code' in df_items.columns:
        mask_service = df_items['Product Type'] == 'service'
        # Asumimos que cuentas de inventario empiezan con '1' o '1-'
        mask_inv_acc = df_items['Inventory Account Code'].astype(str).str.startswith(('1-', '15'), na=False)
        df_items.loc[mask_service & mask_inv_acc, 'ERROR_CRITICO'] = True

    # Filtrar errores para el Excel
    errores_items = df_items[
        (df_items['ESTADO_SKU'] != 'OK') | (df_items['ERROR_CRITICO'] == True)
    ][['Item Name', 'SKU', 'Product Type', 'ESTADO_SKU', 'ERROR_CRITICO', 'Status']]

    audit_findings['ERRORES_ITEMS'] = errores_items
    stats['invalid_skus'] = len(df_items[df_items['ESTADO_SKU'] != 'OK'])
    stats['critical_errors'] = len(df_items[df_items['ERROR_CRITICO'] == True])

# --- B. AUDITORÍA CONTABLE (CoA) ---
if 'COA' in data_map:
    df_coa = data_map['COA']
    # Buscamos formatos inconsistentes (diferentes a X-XXXX)
    def validate_coa(code):
        if pd.isna(code): return 'SIN_CODIGO'
        if not re.match(r'^\d-\d{4}$', str(code).strip()): return 'FORMATO_INCORRECTO'
        return 'OK'

    df_coa['REVISION'] = df_coa['Account Code'].apply(validate_coa)
    audit_findings['ERRORES_COA'] = df_coa[df_coa['REVISION'] != 'OK'][['Account Name', 'Account Code', 'Account Type', 'REVISION']]
    stats['coa_warnings'] = len(audit_findings['ERRORES_COA'])

# ==============================================================================
# 4️⃣ GENERACIÓN DE SALIDAS (REPORTING)
# ==============================================================================

# A. Generar Excel Maestro (Para el equipo técnico)
excel_path = os.path.join(OUTPUT_PATH, 'AUDITORIA_TECNICA_ZOHO.xlsx')
with pd.ExcelWriter(excel_path) as writer:
    for sheet, df in audit_findings.items():
        df.to_excel(writer, sheet_name=sheet, index=False)
print(f"\n💾 Excel Técnico generado: {excel_path}")

# B. Generar Reporte Narrativo Markdown (Para Gerencia/Stakeholders)
md_path = os.path.join(OUTPUT_PATH, 'REPORTE_EJECUTIVO_MIGRACION.md')
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")

markdown_content = f"""
# 📑 REPORTE DE INGENIERÍA DE DATOS: GRUPO ANGEL
**Fecha:** {timestamp}
**Origen:** Auditoría Automática (Python ETL)
**Alcance:** Validación Pre-Migración (Zoho Books -> Supabase)

---

## 1. RESUMEN EJECUTIVO
Se ha realizado un escaneo profundo de la estructura de datos de **MF World S.A.** para garantizar la integridad en la migración al nuevo ecosistema digital. El objetivo es transformar datos contables "crudos" en datos de ingeniería listos para BigQuery.

### 📊 Hallazgos Principales
* **Items Analizados:** {len(data_map.get('ITEMS', []))}
* **SKUs Inválidos/Faltantes:** {stats['invalid_skus']} (Requieren corrección para ser leídos por SQL).
* **Errores Críticos de Configuración:** {stats['critical_errors']} (Items de servicio que afectan inventario erróneamente).
* **Cuentas Contables fuera de Estándar:** {stats['coa_warnings']} (Códigos que no siguen el formato X-XXXX).

---

## 2. POR QUÉ HICIMOS ESTO (BENEFICIOS)
Este proceso de limpieza no es estético, es **funcional**:
1.  **Consolidación Automatizada:** Al estandarizar los SKUs y Cuentas, podremos ver las ventas de las 8 empresas en un solo Dashboard en tiempo real.
2.  **Prevención de Errores Fiscales:** Detectamos servicios que estaban moviendo inventario (costo fantasma), evitando inconsistencias ante la DGI.
3.  **Velocidad:** Una base de datos limpia en Supabase permitirá que la futura App opere en milisegundos.

---

## 3. ARCHIVOS PROCESADOS
{chr(10).join(file_log)}

---

## 4. PRÓXIMOS PASOS (PLAN DE ACCIÓN)
Revisar el archivo adjunto `AUDITORIA_TECNICA_ZOHO.xlsx` y ejecutar:
1.  **Limpieza de SKUs:** Asignar códigos a los items marcados como `FALTA_SKU`.
2.  **Corrección Contable:** Modificar el item crítico detectado (ej: *Vivere Articulo Generico*) para que no afecte cuentas de inventario.
3.  **Homologación:** Ajustar el Catálogo de Cuentas al estándar `X-XXXX`.

*Reporte generado automáticamente por ZBooks AI Architecture.*
"""

with open(md_path, "w", encoding='utf-8') as f:
    f.write(markdown_content)

print(f"📝 Reporte Ejecutivo generado: {md_path}")
print("\n✅ ¡PROCESO COMPLETADO! Ve a tu carpeta de Drive para ver los resultados.")